In [1]:
!pip -q install langchain langchain-community
!pip -q install pypdf
!pip -q install sentence_transformers
!pip install openai
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.4 MB/s eta 0:00:00


In [2]:
!pip install tokenizers
!pip install faiss-cpu
!pip -q install unstructured

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 26.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 6.7 MB/s eta 0:00:00


In [3]:
!pip install numpy==1.24.4
!pip install nltk==3.9.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 72.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
faiss-cpu 1.9.0 requires numpy<3.0,>=1.25.0, but you have numpy 1.24.4 which is incompatible.


In [24]:
import sys
import os
import torch
import textwrap
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.embeddings import HuggingFaceEmbeddings

In [3]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
os.environ['OPENAI_API_KEY'] = "sk-45DF-OPENAI_API_KEY"

In [13]:
import requests

In [22]:
URLs=[
    'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb',
    'https://www.mosaicml.com/blog/mpt-7b',
    'https://stability.ai/blog/stability-ai-launches-the-first-of-its-stablelm-suite-of-language-models',
    'https://lmsys.org/blog/2023-03-30-vicuna/'

]

In [25]:
loaders = UnstructuredURLLoader(urls=URLs)
data = loaders.load()

In [26]:
data

[Document(metadata={'source': 'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb'}, page_content='Open in app\n\nSign up\n\nSign in\n\nWrite\n\nSign up\n\nSign in\n\nPaper Review\n\nPaper Review: Llama 2: Open Foundation and Fine-Tuned Chat Models\n\nLlama 2: one of the best open source models\n\nAndrew Lukyanenko\n\nFollow\n\nPublished in\n\nGoPenAI\n\n15 min read\n\nJul 20, 2023\n\n--\n\nProject link\n\nModel link\n\nPaper link\n\nThe authors of the work present Llama 2, an assortment of pretrained and fine-tuned large language models (LLMs) with sizes varying from 7 billion to 70 billion parameters. The fine-tuned versions, named Llama 2-Chat, are specifically designed for dialogue applications. These models surpass the performance of existing open-source chat models on most benchmarks, and according to human evaluations for usefulness and safety, they could potentially replace closed-source models. The authors also detail their ap

In [27]:
len(data)

4

#**4: Split the Text into Chunks**

In [29]:
text_splitter = CharacterTextSplitter(separator='\n', chunk_size=1000, chunk_overlap=200)

In [32]:
text_chunks = text_splitter.split_documents(data)

In [33]:
len(text_chunks)

86

In [34]:
text_chunks[0]

Document(metadata={'source': 'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb'}, page_content='Open in app\nSign up\nSign in\nWrite\nSign up\nSign in\nPaper Review\nPaper Review: Llama 2: Open Foundation and Fine-Tuned Chat Models\nLlama 2: one of the best open source models\nAndrew Lukyanenko\nFollow\nPublished in\nGoPenAI\n15 min read\nJul 20, 2023\n--\nProject link\nModel link\nPaper link\nThe authors of the work present Llama 2, an assortment of pretrained and fine-tuned large language models (LLMs) with sizes varying from 7 billion to 70 billion parameters. The fine-tuned versions, named Llama 2-Chat, are specifically designed for dialogue applications. These models surpass the performance of existing open-source chat models on most benchmarks, and according to human evaluations for usefulness and safety, they could potentially replace closed-source models. The authors also detail their approach to fine-tuning and safety enhanc

In [35]:
text_chunks[80]

Document(metadata={'source': 'https://lmsys.org/blog/2023-03-30-vicuna/'}, page_content="Figure 3. Response Comparison Assessed by GPT-4\nFigure 3 displays the comparison results between all baselines and Vicuna. GPT-4 prefers Vicuna over state-of-the-art open-source models (LLaMA, Alpaca) in more than 90% of the questions, and it achieves competitive performance against proprietary models (ChatGPT, Bard). In 45% of the questions, GPT-4 rates Vicuna's response as better or equal to ChatGPT's. As GPT-4 assigns a quantitative score to each response on a scale of 10, we calculate the total score for each (baseline, Vicuna) comparison pair by adding up the scores obtained by each model on 80 questions. As shown in Table 2, Vicuna’s total score is 92% of ChatGPT’s. Despite recent advancements, these chatbots still face limitations, such as struggling with basic math problems or having limited coding ability.\nTable 2. Total Scores Assessed by GPT-4.\nBaseline Baseline Score Vicuna Score LLa

In [36]:
embeddings= OpenAIEmbeddings()

<ipython-input-36-b0583f7166b7>:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings= OpenAIEmbeddings()


In [37]:
query_result = embeddings.embed_query("Hello world")
len(query_result)

1536

In [38]:
query_result

[-0.004782061775575981,
 0.004832498948880666,
 -0.01648030318721954,
 -0.024499791856582992,
 -0.017388169512736022,
 0.012634479083701404,
 -0.01921651227200245,
 0.009293024850584886,
 -0.010219805407128945,
 -0.02678206870993028,
 0.02288580789876877,
 0.010320679753738316,
 -0.023629753993121658,
 -0.006582033653962355,
 0.008076231362446179,
 0.0025250044402036543,
 0.025067209474184067,
 -0.01210488992815548,
 0.013151458131013866,
 0.01336581600114345,
 -0.01068634774679803,
 -0.003502221937221743,
 0.0038868043941396877,
 0.008366244294040828,
 -0.020754843030996846,
 -0.0018882366988389192,
 0.012319246866962449,
 -0.018976936979373794,
 0.030665721275503564,
 -0.03127096735515979,
 0.0036882086936406203,
 -0.007748390434457687,
 -0.006083967906354851,
 -0.017791666899173504,
 0.004920763652917884,
 -0.015635483677579894,
 0.001406720554134561,
 -0.015635483677579894,
 0.019758711535781834,
 -0.01607680766342729,
 0.007281847629127293,
 0.008353635117129984,
 0.01159421500363

In [39]:
vectorstore = FAISS.from_documents(text_chunks,embeddings)

In [40]:
llm = ChatOpenAI()

<ipython-input-40-1f2ff8f9afda>:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI()


In [41]:
llm.predict("Please provide summary of Titanic movie")

<ipython-input-41-fa05d6fcb05d>:1: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm.predict("Please provide summary of Titanic movie")


"Titanic is a 1997 epic romance and disaster film directed by James Cameron. The film follows the ill-fated voyage of the RMS Titanic, a luxurious ocean liner that sank in 1912 after hitting an iceberg. The story primarily revolves around the fictional characters of Jack Dawson, a poor artist, and Rose DeWitt Bukater, a wealthy young woman who are drawn together despite their different social backgrounds. As they fall in love, they must navigate the class divisions and disapproval of Rose's fiancé, Cal Hockley. The film alternates between the present-day story of an elderly Rose recounting her experience aboard the Titanic and the events of the doomed voyage. The film culminates in the tragic sinking of the ship, during which Jack sacrifices his life to save Rose. Titanic went on to become one of the highest-grossing films of all time and won multiple Academy Awards, including Best Picture."

In [42]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorstore.as_retriever())

In [45]:
result = chain({"question": "How to good is vicuna?"}, return_only_outputs=True)

In [46]:
result['answer']

'Vicuna is capable of generating detailed and well-structured answers, with quality on par with ChatGPT. However, it may have limitations in tasks involving reasoning or mathematics.\n'

In [48]:
result = chain({"question": "How does Llam2 model outforms other models"}, return_only_outputs=True)
result['answer']

'Llama 2 models outperform MPT models of similar size in all categories except code benchmarks, Falcon models of similar size, their Llama 1 counterparts, and all open-source models. Additionally, the Llama 2-Chat models significantly outperform open-source models on both single turn and multi-turn prompts.\n'

In [49]:
result = chain({"question": "can you share me some details about MPT-7b model"}, return_only_outputs=True)
result['answer']

'The MPT-7B model is a high-quality model built by the MosaicML NLP team that matches the quality of LLaMA-7B and outperforms other open-source 7B - 20B models on standard academic tasks. It was fine-tuned using various tools like ShareGPT-Vicuna, HC3, Alpaca, Helpful and Harmless, and Evol-Instruct for conversational tasks and applications. The MPT-7B model is licensed for commercial use, trained on a large amount of data, and can handle extremely long inputs. Additionally, there are three finetuned models based on MPT-7B: MPT-7B-Instruct, MPT-7B-Chat, and MPT-7B-StoryWriter-65k+.\n'

In [50]:
result = chain({"question": "can you share me some details about Virat Kohli"}, return_only_outputs=True)
result['answer']

"I don't know. \n"

In [54]:
import sys

while True:
    query = input("Enter your query (type 'exit' to quit): ").strip()

    if query.lower() == "exit":
        print("Exiting")
        sys.exit()

    if not query:
        print("Empty query. Please enter a valid question.")
        continue

    # Assuming 'chain' is already defined and properly set up
    result = chain({"question": query})
    print(f"Answer: {result['answer']}")


Enter your query (type 'exit' to quit): tell me about claude model
Answer: There is no information available in the provided text about the Claude model.

Enter your query (type 'exit' to quit): tell me details about falcon model
Answer: I don't know.

Enter your query (type 'exit' to quit): can you share me some details about MPT-7b model
Answer: MPT-7B is a model released by MosaicML that matches the quality of LLaMA-7B and outperforms other open source 7B - 20B models on standard academic tasks. It is licensed for commercial use, trained on a large amount of data (1T tokens), and prepared to handle extremely long inputs. There are three finetuned models based on MPT-7B: MPT-7B-Instruct, MPT-7B-Chat, and MPT-7B-StoryWriter-65k+.

Enter your query (type 'exit' to quit): Llam2
Answer: I don't know.

Enter your query (type 'exit' to quit): tell me details about Llama2
Answer: Llama 2 is an assortment of pretrained and fine-tuned large language models with sizes varying from 7 billion to

SystemExit: 

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
